In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import numpy as np
import requests
from talib import MACD, RSI, BBANDS
from datetime import datetime, timedelta, date
from IPython.display import display, HTML
%matplotlib inline
import json
import math
from dateutil import parser
from pytz import timezone
import pytz
import sys
import iso8601
from pandas import DataFrame as df

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
session = requests.session()

In [ ]:
start_day_to_analyze = "2020-07-27"
end_day_to_analyze = "2020-07-28"
symbols = [
    "SOGO",
    "ABUS",
    "AUY",
    "RIOT",
    "HMY",
    "HTBX",
    "HL",
    "KGC",
    "GFI",
    "CAN",
    "EXK",
    "AG",
    "GNCA",
    "PDSB",
    "CAPR",
    "CDE",
    "WHLM",
    "FSM",
    "CX",
    "CANF",
    "DMYT",
    "APHA",
    "BTG",
    "SRNE",
    "PSLV",
    "TCRR",
    "IAG",
    "SVM",
    "SBSW",
    "TRVN",
    "RIGL",
    "UMC",
    "CNXM",
    "AXU",
    "SILJ",
    "FMCI",
    "EGO",
    "BLMN",
    "AIM",
    "BVN",
    "CEMI",
    "SAND",
    "CASI",
    "XPER",
    "TPR",
    "AGI",
    "CGC",
    "ZYNE",
    "PVG",
    "SHLL",
    "EQX",
    "ANY",
    "BSIG",
    "GORO",
    "SOHU",
    "NG",
    "GSS",
    "BLNK",
    "PLG",
    "KTB",
    "DRD",
    "TPH",
    "SLVP",
    "MAG",
    "OR",
    "YCBD",
    "MICT",
    "AMRS",
    "TACO",
    "FTFT",
    "SILV",
    "IPOC",
    "UPWK",
    "AMKR",
    "ASX",
    "MTA",
    "EMX",
    "AUG",
    "SA",
    "OSG",
    "MESO",
    "SQBG",
    "ERESU",
    "ACIU",
    "ADTX",
    "PRTS",
    "TMQ",
    "RESN",
    "IMMR",
    "NPTN",
    "NETE",
    "XNET",
    "SEED",
    "DTIL",
    "EBON",
    "SVVC",
    "HMI",
    "FF",
    "INVA",
    "IMUX",
    "ATOM",
    "GLG",
    "SPPP",
    "KLDO",
    "GECC",
    "HOFV",
    "LEU",
    "STXS",
    "IMTX",
    "HARP",
    "ELA",
    "QH",
    "MTBC",
    "KORU",
    "INZY",
    "MCC",
    "BMRG",
    "RUBY",
    "CLPT",
    "YI",
    "PASG",
    "RENN",
    "MRUS",
    "FMO",
]

In [ ]:
minute_history = {}

for symbol in symbols:
    if symbol not in minute_history:
        minute_history[symbol] = api.polygon.historic_agg_v2(
            symbol, 
            1, 
            'minute',
            _from = str((datetime.strptime(start_day_to_analyze, '%Y-%m-%d')-timedelta(days=10)).date()),
            to=str((datetime.strptime(end_day_to_analyze, '%Y-%m-%d')+timedelta(days=1)).date())).df
        print(f"{symbol} loaded {len(minute_history[symbol].index)} agg data points")


In [ ]:
minute_history

In [ ]:
def grouper(iterable):
    prev = None
    group = []
    for item in iterable:
        if not prev or -0.02 <= float(item - prev) / prev <= 0.02:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group


def find_resistance(current_value: float, minute_history, now):
    """calculate next resistance"""
    est = pytz.timezone("US/Eastern")
    now = pd.Timestamp(now.astimezone(est))
    real_start_time = pd.Timestamp(
        now.to_pydatetime().replace(hour=9, minute=30, second=0, microsecond=0)
    )

    now_index = minute_history["close"].index.get_loc(now, method="nearest")
    start_index = minute_history["close"].index.get_loc(
        real_start_time, method="nearest"
    )

    series = (
        (
            minute_history["close"][start_index : now_index + 1]
            .dropna()
            .between_time("9:30", "16:00")
            .resample("5min")
            .max()
        )
        .between_time("9:30", "16:00")
        .dropna()
    )
    diff = np.diff(series.values)

    high_index = np.where((diff[:-1] >= 0) & (diff[1:] <= 0))[0] + 1
    if len(high_index) > 0:
        if len(high_index) == 1:
            true_max_index = high_index
        else:
            i = 0
            h = high_index[i]
            true_max_index = []
            for local_high in high_index[1:]:
                if 5 * (local_high - high_index[i]) < 60:
                    if series[local_high] > series[h]:
                        h = local_high
                else:
                    true_max_index.append(h)
                    h = local_high
                i += 1
            if len(true_max_index) == 0:
                true_max_index.append(high_index[-1])
            elif series[high_index[-1]] > series[true_max_index[-1]]:
                if 5 * (high_index[-1] - true_max_index[-1]) < 60:
                    true_max_index[-1] = high_index[-1]
                else:
                    true_max_index.append(high_index[-1])
        local_maximas = sorted(
            [series[i] for i in true_max_index if series[i] >= current_value]
        )
        if len(local_maximas) > 0:
            #print()
            #print(
            #    series,
            #    [(series[i], series.index[i]) for i in high_index],
            #
            #[(series[i], series.index[i]) for i in true_max_index],
            #)
            return local_maximas

    return None

In [ ]:
def find_resistance2(current_value: float, minute_history, now):
    """calculate next resistance"""
    est = pytz.timezone("US/Eastern")
    now = pd.Timestamp(now.astimezone(est))

    real_start_time = pd.Timestamp(
        now.to_pydatetime().replace(hour=9, minute=30, second=0, microsecond=0)
        - timedelta(days=2)
    )

    now_index = minute_history["close"].index.get_loc(now, method="nearest")
    start_index = minute_history["close"].index.get_loc(
        real_start_time, method="nearest"
    )

    series = (
        (
            minute_history["close"][start_index : now_index + 1]
            .dropna()
            .between_time("9:30", "16:00")
            .resample("5min")
            .max()
        )
        .between_time("9:30", "16:00")
        .dropna()
    )
    diff = np.diff(series.values)

    high_index = np.where((diff[:-1] >= 0) & (diff[1:] <= 0))[0] + 1
    if len(high_index) > 0:
        local_maximas = sorted(
            [series[i] for i in high_index if series[i] >= current_value]
        )

        if len(local_maximas) > 0:
            g = grouper(local_maximas)
            
            rc = []
            for group in g:
                if len(group) >= 3:
                    #print(group)
                    rc.append(max(group))
            rc = sorted(rc)
            # print(now, rc)
            # print()
            # print(
            #    series,
            #    [(series[i], series.index[i]) for i in high_index],
            #
            # [(series[i], series.index[i]) for i in true_max_index],
            # )
            return rc if len(rc) > 0 else None

    return None

In [ ]:
for symbol in minute_history:
    start_date = datetime.strptime(start_day_to_analyze, "%Y-%m-%d")
    start_date = start_date.replace(hour=9, minute=30)
    end_date = start_date.replace(hour=16, minute=0)
    start_index = minute_history[symbol]["close"].index.get_loc(
        start_date, method="nearest"
    )
    start_end = minute_history[symbol]["close"].index.get_loc(
        end_date, method="nearest"
    )
    print(symbol)
    minute_history[symbol]["resistance"] = minute_history[symbol][
        start_index:start_end
    ].apply(
        lambda x: val[0]
        if (val := find_resistance(x["close"], minute_history[symbol], x.name))
        is not None
        else None,
        axis=1,
    )

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(minute_history['SNAP'][start_index:end_index].between_time(
            "9:30", "16:00"
        ))

In [ ]:
start_date = datetime.strptime(start_day_to_analyze, "%Y-%m-%d")
start_date = start_date.replace(hour=9, minute=30)
end_date = start_date.replace(hour=16, minute=0)
for symbol in minute_history:
    start_index = minute_history[symbol]["close"].index.get_loc(
        start_date, method="nearest"
    )
    end_index = minute_history[symbol]["close"].index.get_loc(
        end_date, method="nearest"
    )

    start_date = datetime.strptime(start_day_to_analyze, "%Y-%m-%d")
    start_date = start_date.replace(hour=9, minute=30)
    print(start_date)
    plt.plot(
        minute_history[symbol]["close"][start_index:end_index].between_time(
            "9:30", "16:00"
        ),
        label=symbol,
    )
    plt.plot(
        minute_history[symbol]["close"][start_index:end_index]
        .between_time("9:30", "16:00")
        .resample("5min")
        .max(),
        label="resampled",
    )
    plt.xticks(rotation=45)

    for index, row in minute_history[symbol][start_index:end_index].iterrows():
        if not math.isnan(row["resistance"]):
            plt.scatter(row.name.to_pydatetime(), row["resistance"], c="r", s=10)

    # resistances = minute_history[symbol].resistance.dropna().unique().tolist()
    # if resistances:
    #    for r in resistances:
    #        plt.axhline(y=r, color="r")
    plt.legend()
    plt.show()